# 5章 顧客の退会を予測する１０本ノック

引き続き、スポーツジムの会員データを使って顧客の行動を分析していきます。  
３章では顧客の全体像を把握し、4章では数ヶ月利用している顧客の来月の利用回数の予測を行いました。   
ここでは、教師あり学習の分類を用いて、顧客の退会予測を取り扱います。

In [5]:
import warnings
warnings.filterwarnings('ignore')

### ノック41：データを読み込んで利用データを整形しよう

In [31]:
import pandas as pd
customer = pd.read_csv('customer_join.csv')
uselog_months = pd.read_csv('use_log_months.csv')
customer.head()

,customer_id,name,class,gender,start_date,end_date,campaign_id,is_deleted,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,OA832399,XXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,5.0,8,2,1,2019-04-30,47
1,PL270116,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,5.083333,5.0,7,3,1,2019-04-30,47
2,OA974876,XXXXX,C01,M,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.583333,5.0,6,3,1,2019-04-30,47
3,HD024127,XXXXX,C01,F,2015-05-01,NaN,CA1,0,オールタイム,10500,通常,4.833333,4.5,7,2,1,2019-04-30,47
4,HD661448,XXXXX,C03,F,2015-05-01,NaN,CA1,0,ナイト,6000,通常,3.916667,4.0,6,1,1,2019-04-30,47


In [11]:
# 当月+過去1ヶ月分の利用回数を集計
year_months = list(uselog_months["年月"].unique())
uselog = pd.DataFrame()

for i in range(1, len(year_months)):
    tmp = uselog_months.loc[uselog_months["年月"]==year_months[i]]
    tmp.rename(columns={"count":"count_0"}, inplace=True)
    tmp_before = uselog_months.loc[uselog_months["年月"]==year_months[i-1]]
    del tmp_before["年月"]
    tmp_before.rename(columns={"count":"count_1"}, inplace=True)
    tmp = pd.merge(tmp, tmp_before, on="customer_id", how="left")
    uselog = pd.concat([uselog, tmp], ignore_index=True)

uselog.head()

,年月,customer_id,count_0,count_1
0,201805,AS002855,5,4.0
1,201805,AS009373,4,3.0
2,201805,AS015233,7,NaN
3,201805,AS015315,3,6.0
4,201805,AS015739,5,7.0


### ノック42：退会前月の退会顧客データを作成しよう

In [40]:
from dateutil.relativedelta import relativedelta

# 退会したユーザー
exit_customer = customer.loc[customer["is_deleted"]==1]
exit_customer["exit_date"] = None
exit_customer["end_date"] = pd.to_datetime(exit_customer["end_date"])

# 退会前月(退会申請月)
for i in range(len(exit_customer)):
    exit_customer["exit_date"].iloc[i] = exit_customer["end_date"].iloc[i] - relativedelta(months=1)

exit_customer["exit_date"] = pd.to_datetime(exit_customer["exit_date"], format='%Y-%m-%d')
exit_customer["年月"] = exit_customer["exit_date"].dt.strftime("%Y%m")
uselog["年月"] = uselog["年月"].astype(str)

exit_uselog = pd.merge(uselog, exit_customer, on=["customer_id", "年月"], how="left")
print(len(uselog))
exit_uselog.head()

33851


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201805,AS002855,5,4.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
1,201805,AS009373,4,3.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
2,201805,AS015233,7,NaN,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
3,201805,AS015315,3,6.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
4,201805,AS015739,5,7.0,NaN,NaN,NaN,NaN,NaT,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT


In [41]:
# uselogベースで退会ユーザーのみ結合してるので欠損値が多い→消す
exit_uselog = exit_uselog.dropna(subset=["name"])
print(len(exit_uselog))
print(len(exit_uselog["customer_id"].unique()))
exit_uselog.head()

1104
1104


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
19,201805,AS055680,3,3.0,XXXXX,C01,M,2018-03-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,3.0,3.0,0.0,2018-06-30,3.0,2018-05-30
57,201805,AS169823,2,3.0,XX,C01,M,2017-11-01,2018-06-30,CA1,...,10500.0,通常,3.000000,3.0,4.0,2.0,1.0,2018-06-30,7.0,2018-05-30
110,201805,AS305860,5,3.0,XXXX,C01,M,2017-06-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,12.0,2018-05-30
128,201805,AS363699,5,3.0,XXXXX,C01,M,2018-02-01,2018-06-30,CA1,...,10500.0,通常,3.333333,3.0,5.0,2.0,0.0,2018-06-30,4.0,2018-05-30
147,201805,AS417696,1,4.0,XX,C03,F,2017-09-01,2018-06-30,CA1,...,6000.0,通常,2.000000,1.0,4.0,1.0,0.0,2018-06-30,9.0,2018-05-30


### ノック43：継続顧客のデータを作成しよう

In [42]:
# 継続顧客のデータ
conti_customer = customer.loc[customer["is_deleted"]==0]
# uselogベースの結合(どの月のデータも使用できるのでidのみで結合)
conti_uselog = pd.merge(uselog, conti_customer, on=["customer_id"], how="left")
print(len(conti_uselog))
# 欠損値の除去
conti_uselog = conti_uselog.dropna(subset=["name"])
print(len(conti_uselog))

33851
27422


In [45]:
# 顧客あたりデータが1件になるように調整(継続顧客が多すぎるため)
# アンダーサンプリング(データの不均衡時、多数側を減らして調整すること)

# 全体のデータをサンプリング(シャッフル)
conti_uselog = conti_uselog.sample(frac=1).reset_index(drop=True)
# 重複削除
conti_uselog = conti_uselog.drop_duplicates(subset="customer_id")
print(len(conti_uselog))
conti_uselog.head()

2842


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,class_name,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period
0,201901,PL001979,4,5.0,XXXXX,C01,M,2016-06-01,NaN,CA1,...,オールタイム,10500.0,通常,4.750000,5.0,7.0,3.0,1.0,2019-04-30,34.0
1,201807,TS548118,5,8.0,XXXXXX,C01,M,2015-09-01,NaN,CA1,...,オールタイム,10500.0,通常,4.416667,4.0,8.0,1.0,1.0,2019-04-30,43.0
2,201901,OA190016,8,9.0,XXXX,C01,M,2018-09-05,NaN,CA1,...,オールタイム,10500.0,通常,8.000000,8.0,9.0,7.0,1.0,2019-04-30,7.0
3,201812,PL505814,3,3.0,XXXX,C02,F,2016-01-01,NaN,CA1,...,デイタイム,7500.0,通常,4.666667,5.0,6.0,3.0,1.0,2019-04-30,39.0
4,201901,GD110372,6,5.0,XXXXXX,C03,F,2017-10-01,NaN,CA1,...,ナイト,6000.0,通常,6.500000,6.5,9.0,4.0,1.0,2019-04-30,18.0


In [46]:
# 継続顧客と退会顧客のデータを縦結合
predict_data = pd.concat([conti_uselog, exit_uselog],ignore_index=True)
print(len(predict_data))
predict_data.head()

3946


,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,price,campaign_name,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date
0,201901,PL001979,4,5.0,XXXXX,C01,M,2016-06-01,NaN,CA1,...,10500.0,通常,4.750000,5.0,7.0,3.0,1.0,2019-04-30,34.0,NaT
1,201807,TS548118,5,8.0,XXXXXX,C01,M,2015-09-01,NaN,CA1,...,10500.0,通常,4.416667,4.0,8.0,1.0,1.0,2019-04-30,43.0,NaT
2,201901,OA190016,8,9.0,XXXX,C01,M,2018-09-05,NaN,CA1,...,10500.0,通常,8.000000,8.0,9.0,7.0,1.0,2019-04-30,7.0,NaT
3,201812,PL505814,3,3.0,XXXX,C02,F,2016-01-01,NaN,CA1,...,7500.0,通常,4.666667,5.0,6.0,3.0,1.0,2019-04-30,39.0,NaT
4,201901,GD110372,6,5.0,XXXXXX,C03,F,2017-10-01,NaN,CA1,...,6000.0,通常,6.500000,6.5,9.0,4.0,1.0,2019-04-30,18.0,NaT


### ノック44：予測する月の在籍期間を作成しよう

In [50]:
predict_data["period"] = 0
predict_data["now_date"] = pd.to_datetime(predict_data["年月"], format="%Y%m")
predict_data["start_date"] = pd.to_datetime(predict_data["start_date"])

for i in range(len(predict_data)):
    delta = relativedelta(predict_data["now_date"][i], predict_data["start_date"][i])
    predict_data["period"][i] = int(delta.years*12 + delta.months)

predict_data.head()

,年月,customer_id,count_0,count_1,name,class,gender,start_date,end_date,campaign_id,...,mean,median,max,min,routine_flg,calc_date,membership_period,exit_date,period,now_date
0,201901,PL001979,4,5.0,XXXXX,C01,M,2016-06-01,NaN,CA1,...,4.750000,5.0,7.0,3.0,1.0,2019-04-30,34.0,NaT,31,2019-01-01
1,201807,TS548118,5,8.0,XXXXXX,C01,M,2015-09-01,NaN,CA1,...,4.416667,4.0,8.0,1.0,1.0,2019-04-30,43.0,NaT,34,2018-07-01
2,201901,OA190016,8,9.0,XXXX,C01,M,2018-09-05,NaN,CA1,...,8.000000,8.0,9.0,7.0,1.0,2019-04-30,7.0,NaT,3,2019-01-01
3,201812,PL505814,3,3.0,XXXX,C02,F,2016-01-01,NaN,CA1,...,4.666667,5.0,6.0,3.0,1.0,2019-04-30,39.0,NaT,35,2018-12-01
4,201901,GD110372,6,5.0,XXXXXX,C03,F,2017-10-01,NaN,CA1,...,6.500000,6.5,9.0,4.0,1.0,2019-04-30,18.0,NaT,15,2019-01-01


### ノック45：欠損値を除去しよう

In [48]:
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1               260
name                    0
class                   0
gender                  0
start_date              0
end_date             2842
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2842
period                  0
now_date                0
dtype: int64

In [53]:
predict_data = predict_data.dropna(subset=['count_1'])
predict_data.isna().sum()

年月                      0
customer_id             0
count_0                 0
count_1                 0
name                    0
class                   0
gender                  0
start_date              0
end_date             2634
campaign_id             0
is_deleted              0
class_name              0
price                   0
campaign_name           0
mean                    0
median                  0
max                     0
min                     0
routine_flg             0
calc_date               0
membership_period       0
exit_date            2634
period                  0
now_date                0
dtype: int64

### ノック46：文字列型の変数を処理できるように整形しよう

In [54]:
target_col = ["campaign_name", "class_name", "gender", "count_1", "routine_flg", "period", "is_deleted"]
predict_data = predict_data[target_col]
predict_data.head()

,campaign_name,class_name,gender,count_1,routine_flg,period,is_deleted
0,通常,オールタイム,M,5.0,1.0,31,0.0
1,通常,オールタイム,M,8.0,1.0,34,0.0
2,通常,オールタイム,M,9.0,1.0,3,0.0
3,通常,デイタイム,F,3.0,1.0,35,0.0
4,通常,ナイト,F,5.0,1.0,15,0.0


In [55]:
# カテゴリカル変数→ダミー変数
predict_data = pd.get_dummies(predict_data)
predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,campaign_name_通常,class_name_オールタイム,class_name_デイタイム,class_name_ナイト,gender_F,gender_M
0,5.0,1.0,31,0.0,0,0,1,1,0,0,0,1
1,8.0,1.0,34,0.0,0,0,1,1,0,0,0,1
2,9.0,1.0,3,0.0,0,0,1,1,0,0,0,1
3,3.0,1.0,35,0.0,0,0,1,0,1,0,1,0
4,5.0,1.0,15,0.0,0,0,1,0,0,1,1,0


In [56]:
# gender_Fにフラグが立っていない = 男性 なので不要な列
del predict_data["gender_M"]
# 3項目の際も同じく、2列とも0なら残りの1つと判断できる
del predict_data["campaign_name_通常"]
del predict_data["class_name_ナイト"]

predict_data.head()

,count_1,routine_flg,period,is_deleted,campaign_name_入会費半額,campaign_name_入会費無料,class_name_オールタイム,class_name_デイタイム,gender_F
0,5.0,1.0,31,0.0,0,0,1,0,0
1,8.0,1.0,34,0.0,0,0,1,0,0
2,9.0,1.0,3,0.0,0,0,1,0,0
3,3.0,1.0,35,0.0,0,0,0,1,1
4,5.0,1.0,15,0.0,0,0,0,0,1


### ノック47：決定木を用いて退会予測モデルを作成してみよう

In [60]:
# 決定木 直感的なのでよく最初に試される
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

exit = predict_data.loc[predict_data["is_deleted"]==1]
# 退会顧客と同数の継続顧客をランダムでサンプリング
conti = predict_data.loc[predict_data["is_deleted"]==0].sample(len(exit))

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = train_test_split(X,y)

model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print(y_test_pred)

[0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0.
 1. 1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1.
 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1.
 0. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0.
 1. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 0. 1.
 0. 0. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 0. 1.
 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1.
 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 0. 0. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 0.
 0. 1. 1. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0.
 0. 0. 0. 0. 1. 0. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1.
 0. 1. 1. 0. 1. 0. 1. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 0.
 0. 0. 0. 0. 0. 0. 1. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0.

In [61]:
results_test = pd.DataFrame({"y_test":y_test ,"y_pred":y_test_pred })
results_test.head()

,y_test,y_pred
1186,0.0,0.0
790,1.0,1.0
1603,0.0,0.0
800,1.0,1.0
972,1.0,1.0


### ノック48：予測モデルの評価を行ない、モデルのチューニングをしてみよう

In [62]:
# 予測の正解件数
correct = len(results_test.loc[results_test["y_test"]==results_test["y_pred"]])
score_test = correct / len(results_test)
print(score_test)

0.8935361216730038


In [63]:
# 上記の予測を行える関数
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.8935361216730038
0.9746514575411914


In [64]:
# 過学習気味なので、チューニングの必要がある
# データを増やす、変数を見直す、モデルパラメータの変更等...

X = pd.concat([exit, conti], ignore_index=True)
y = X["is_deleted"]
del X["is_deleted"]
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y)

# 決定木の深さの上限を5に設定
model = DecisionTreeClassifier(random_state=0, max_depth=5)
model.fit(X_train, y_train)
print(model.score(X_test, y_test))
print(model.score(X_train, y_train))

0.9201520912547528
0.9258555133079848


### ノック49：モデルに寄与している変数を確認しよう

In [65]:
# 決定木における重要変数の取得
# 木構造を可視化することも可能
importance = pd.DataFrame({"feature_names":X.columns, "coefficient":model.feature_importances_})
importance

,feature_names,coefficient
0,count_1,0.354919
1,routine_flg,0.139395
2,period,0.502691
3,campaign_name_入会費半額,0.002774
4,campaign_name_入会費無料,0.000000
5,class_name_オールタイム,0.000000
6,class_name_デイタイム,0.000220
7,gender_F,0.000000


### ノック50：顧客の退会を予測しよう

In [66]:
# 予測したいデータの作成
count_1 = 3
routine_flg = 1
period = 10
campaign_name = "入会費無料"
class_name = "オールタイム"
gender = "M"

In [69]:
if campaign_name == "入会費半額":
    campaign_name_list = [1, 0]
elif campaign_name == "入会費無料":
    campaign_name_list = [0, 1]
elif campaign_name == "通常":
    campaign_name_list = [0, 0]
if class_name == "オールタイム":
    class_name_list = [1, 0]
elif class_name == "デイタイム":
    class_name_list = [0, 1]
elif class_name == "ナイト":
    class_name_list = [0, 0]
if gender == "F":
    gender_list = [1]
elif gender == "M":
    gender_list = [0]
input_data = [count_1, routine_flg, period]
input_data.extend(campaign_name_list)
input_data.extend(class_name_list)
input_data.extend(gender_list)
input_data

[3, 1, 10, 0, 1, 1, 0, 0]

In [72]:
# 退会予測
print(model.predict([input_data]))
# 退会確率予測(継続:退会)
print(model.predict_proba([input_data]))

[1.]
[[0.008 0.992]]
